In [1]:
import numpy as np
import pandas as pd

In [2]:
## File paths
data_path = 'C:/Users\lukec\PycharmProjects\petrochemical-data\data\combined\icisFacilityEmissions_ihsWeighted_w_uncertainties_allgases_fert2050_3.parquet'

save_path = "C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\data\classification_inputs/petrochemicals"

In [3]:
def filter_df(df:pd.DataFrame, cols:list, filters:list) -> pd.DataFrame:
    """Function for filtering a pandas dataframe based on column values"""
    for col, filt in zip(cols, filters):
        df = df[[i in filt for i in df[col]]]
    return df

In [4]:
## Import and manipulate facility data
facilities = pd.read_parquet(data_path)

# Use only one target
gas_t, type_t = 'CO2e_100a', ['Feedstock', 'Organic chemicals', 'Primary chemicals', 'Other intermediates', 'Direct Utilities', 'Indirect Utilities', 'Direct Process']
used = filter_df(facilities, ['Gas', 'Type'], [[gas_t], type_t])

# Get correct columns
columns = ['PRODUCT', 'COUNTRY/TERRITORY', 'COMPANY', 'ROUTE', 'TECHNOLOGY','SITE', '#', 'COMPLEX', 'LICENSOR', 'START_YR', 'LATITUDE', 'LONGITUDE']
used = used.groupby(columns).sum().reset_index()

years = [str(i) for i in range(1978, 2051)]
used = used[columns+years]

# Define start year as first year with value if unspecified + find end year
used['START_YR'] = np.where(used['START_YR'].replace('n.a.', 0).astype(float).lt(1), used[years].transpose().ne(0).idxmax(), used['START_YR']).astype(float)
used['END_YR'] = used[list(reversed(years))].transpose().ne(0).idxmax().astype(float)

# Only take actual facilities and not country fillers
used = used[[i not in ['n.a.', '~ADDITIONAL'] for i in used['COMPANY']]]
used[['LATITUDE', 'LONGITUDE']+years] = used[['LATITUDE', 'LONGITUDE']+years].astype(float)
used = used.drop_duplicates()

used.head()

,PRODUCT,COUNTRY/TERRITORY,COMPANY,ROUTE,TECHNOLOGY,SITE,#,COMPLEX,LICENSOR,START_YR,...,2042,2043,2044,2045,2046,2047,2048,2049,2050,END_YR
0,"1,4-BUTANEDIOL",BELGIUM,BASF,MALEIC ANHYDRIDE,HYDRATION,FELUY,1.0,n.a.,KVAERNER PROCESS TECH,1997.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2007.0
1,"1,4-BUTANEDIOL",CHINA,BLUESTAR NEW CHEM. MAT.,MALEIC ANHYDRIDE,HYDRATION,NANJING,1.0,n.a.,KVAERNER PROCESS TECH,2009.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2016.0
2,"1,4-BUTANEDIOL",CHINA,BLUESTAR NEW CHEM. MAT.,MALEIC ANHYDRIDE,HYDRATION,NANJING,2.0,n.a.,n.a.,2012.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2016.0
3,"1,4-BUTANEDIOL",CHINA,CHANGLIAN CHEMICAL (PANJIN),PROPYLENE OXIDE,VIA ALLYL ALCOHOL,PANJIN,1.0,0,n.a.,2014.0,...,297.751361,305.600962,312.993304,320.309437,327.62557,334.941702,342.257835,349.573968,356.890101,2050.0
4,"1,4-BUTANEDIOL",CHINA,CHANGZHOU CHEMICAL PLANT,MALEIC ANHYDRIDE,HYDRATION,CHANGZHOU,1.0,n.a.,SINO,1998.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2008.0


In [5]:
def convert_to_yearly(df, columns, years) -> pd.DataFrame:
    """Convert dataframe of plants with entry for each year into dataframe with row for each year"""
    # Convert to yearly
    yearly = pd.melt(df, id_vars=columns+['END_YR'], value_vars=years, var_name='Year', value_name='Emissions')

    # Get rid of emissions for years before start year
    yearly['Age'] = yearly['Year'].astype(int) - yearly['START_YR'].astype(int)
    yearly = yearly[yearly['Age'] >= 0]

    # Get rid of emissions for years after end years
    yearly['ToGo'] = yearly['END_YR'].astype(int) - yearly['Year'].astype(int)
    yearly = yearly[yearly['ToGo'] >= 0]

    return yearly.drop(columns=['ToGo'])

used_pivoted = convert_to_yearly(used, columns, years)

In [6]:
used.to_csv(save_path+'.csv', index=False)
used_pivoted.to_csv(save_path+'_melted.csv', index=False)

In [3]:
data = pd.read_csv(save_path+'.csv')
data

,PRODUCT,COUNTRY/TERRITORY,COMPANY,ROUTE,TECHNOLOGY,SITE,#,COMPLEX,LICENSOR,START_YR,...,2042,2043,2044,2045,2046,2047,2048,2049,2050,END_YR
0,"1,4-BUTANEDIOL",BELGIUM,BASF,MALEIC ANHYDRIDE,HYDRATION,FELUY,1.0,n.a.,KVAERNER PROCESS TECH,1997.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2007.0
1,"1,4-BUTANEDIOL",CHINA,BLUESTAR NEW CHEM. MAT.,MALEIC ANHYDRIDE,HYDRATION,NANJING,1.0,n.a.,KVAERNER PROCESS TECH,2009.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.0
2,"1,4-BUTANEDIOL",CHINA,BLUESTAR NEW CHEM. MAT.,MALEIC ANHYDRIDE,HYDRATION,NANJING,2.0,n.a.,n.a.,2012.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.0
3,"1,4-BUTANEDIOL",CHINA,CHANGLIAN CHEMICAL (PANJIN),PROPYLENE OXIDE,VIA ALLYL ALCOHOL,PANJIN,1.0,0,n.a.,2014.0,...,297.751361,305.600962,312.993304,320.309437,327.625570,334.941702,342.257835,349.573968,356.890101,2050.0
4,"1,4-BUTANEDIOL",CHINA,CHANGZHOU CHEMICAL PLANT,MALEIC ANHYDRIDE,HYDRATION,CHANGZHOU,1.0,n.a.,SINO,1998.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29159,VINYL ACETATE M.,UNITED STATES,CELANESE,ETHYLENE,ACETOXYLATION,CLEAR LAKE/TX,1.0,0,ICI,1978.0,...,674.553535,672.120693,669.770742,667.499516,665.303125,663.177930,661.120524,659.127714,657.196506,2050.0
29160,VINYL ACETATE M.,UNITED STATES,DOW CHEMICAL,ETHYLENE,ACETOXYLATION,TEXAS CITY/TX,1.0,*,BAYER,1976.0,...,547.137867,545.164562,543.258491,541.416274,539.634757,537.910987,536.242202,534.625813,533.059388,2050.0
29161,VINYL ACETATE M.,UNITED STATES,INEOS ACETYLS,ETHYLENE,n.a.,~UNKNOWN USA,1.0,0,n.a.,2026.0,...,449.702357,448.080462,446.513828,444.999678,443.535417,442.118620,440.747016,439.418476,438.131004,2050.0
29162,VINYL ACETATE M.,UNITED STATES,KURARAY CO.,ETHYLENE,ACETOXYLATION,LA PORTE/TX,1.0,n.a.,n.a.,1978.0,...,502.167631,500.356516,498.607108,496.916307,495.281215,493.699125,492.167501,490.683965,489.246288,2050.0


In [11]:
data['COMPLEX'].unique()

array(['n.a.', '0', '*', 'REF PULAU BUKO SHELL EPPL',
       'REF GREAT FALL MONTANA RE', 'AROMATICS COMPLX NO. 0',
       'AROMATICS COMPLEX NO. 1', 'AROMATICS COMPLX NO. 2',
       'AROMATICS COMPLX NO. 3', 'AROMATICS COMPLX NO. 6',
       'REF STRATHCONA IMPERIAL O', 'REF CILACAP PERTAMINA',
       'REF PLOCK      MZP', 'REF LYSEKIL    SCANRAFF',
       'REF COFFEYVILL FARMLAND I', 'REF ARTESIA    NAVAJO REF',
       'REF SINCLAIR SINCLAIR O', 'REF CHEYENNE FRONTIER R',
       'REF DICKINSON NORTHLAND', 'REF MOBILE LOUISIANA',
       'REF IZMIR      TURKISH PE', 'REF PLOIESTI   PETROBRAZI',
       'REF PULAU MERL SINGAP.REF'], dtype=object)